In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
import glob
import gc

import torch
import xarray as xr
from neuralhydrology.nh_run import start_run, eval_run
from scripts.file_manipulator import file_rewriter

from pathlib import Path
import geopandas as gpd

import random
random.seed(42)
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
ts_dir = Path('../geo_data/time_series')

### Run for different configs


#### ERA5 precipitation

##### train

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5', 't_min_e5', 'prcp_e5'])

if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/era5_static_qmm.yml"))


##### test

In [ ]:
run_dir = Path("./runs_q_mm/era5_prcp_0404_074943/")
eval_run(run_dir=run_dir, period="test")

#### ERA5-Land precipitation

##### train

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])
if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/era5L_static_qmm.yml"))

##### test

In [ ]:
run_dir = Path("./runs_static/era5L_static_prcp_0704_170903/")
eval_run(run_dir=run_dir, period="test", epoch=21)

#### GPCP precipitation

##### train

In [ ]:
# gpcp
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5', 't_min_e5', 'prcp_gpcp'])
if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/gpcp_static_qmm.yml"))
gc.collect()

##### test

In [ ]:
run_dir = Path("./runs_q_cms/gpcp_prcp_3003_063422")
eval_run(run_dir=run_dir, period="test")

#### IMERG precipitation

##### train

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5', 't_min_e5', 'prcp_imerg'])
if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/imerg_static_qmm.yml"))
gc.collect()


##### test

In [ ]:
run_dir = Path("./runs_q_cms/imerg_prcp_3003_135405")
eval_run(run_dir=run_dir, period="test")

#### MSWEP precipitation

##### train

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5', 't_min_e5', 'prcp_mswep'])
if torch.cuda.is_available():
    start_run(config_file=Path("./no_static_configs/mswep_qmm.yml"))
gc.collect()

##### static run

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_mswep'])
if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/mswep_static_qmm.yml"))
gc.collect()

##### test

In [ ]:
run_dir = Path("./runs_q_cms/mswep_prcp_3103_073833")
eval_run(run_dir=run_dir, period="test")

#### Level

In [14]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_h/*.nc'),
              ts_dir=ts_dir,
              hydro_target='lvl_mbs',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])
if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/era5L_static_level_mbs.yml"))
gc.collect()

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_h/*.nc'),
              ts_dir=ts_dir,
              hydro_target='lvl_sm',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])
if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/era5L_static_level_lvl_relative.yml"))
gc.collect()

### Blind forecast

In [ ]:
partial_gauges = gpd.read_file(
    '../geo_data/great_db/geometry/gauges_partial_q.gpkg')

In [ ]:
with open('./basins_test.txt', 'w') as the_file:
    for gauge_name in partial_gauges['gauge_id']:
        the_file.write(f'{int(gauge_name)}\n')
# add train
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])
# add test
q_pathes = glob.glob('../geo_data/great_db/nc_concat/*.nc')
for file in q_pathes:
    gauge_id = file.split('/')[-1][:-3]
    if gauge_id in list(partial_gauges['gauge_id']):
        ds = xr.open_dataset(file)
        filename = file.split('/')[-1]
        try:
            ds = ds.drop('gauge_id')
            ds.to_netcdf(f'{ts_dir}/{filename}')
        except ValueError:
            continue
        

if torch.cuda.is_available():
    start_run(config_file=Path("./static_configs/era5L_static_blind.yml"))
gc.collect()

In [ ]:
# add test
q_pathes = glob.glob('../geo_data/great_db/nc_concat/*.nc')
for file in q_pathes:
    gauge_id = file.split('/')[-1][:-3]
    if gauge_id in list(partial_gauges['gauge_id']):
        ds = xr.open_dataset(file)
        filename = file.split('/')[-1]
        try:
            ds = ds.drop('gauge_id')
            ds = ds.rename({'index': 'date'})
            ds.to_netcdf(f'{ts_dir}/{filename}')
        except ValueError:
            continue
# for gauge in partial_gauges['gauge_id']:
    
#     ds = xr.open_dataset(f'../geo_data/time_series/{gauge}.nc')
    
#     ds.to_netcdf(f'../geo_data/time_series/{gauge}.nc')
    
    

In [ ]:
run_dir = Path("./runs_static/era5L_static_prcp_0904_152029/")
eval_run(run_dir=run_dir, period="test", epoch=24)

### Bad regions

#### Kavkaz

In [11]:
gauges = gpd.read_file(
    '../paper_visualisation/poor_gauges/kavkaz.gpkg')
# add test
q_pathes = glob.glob('../geo_data/great_db/nc_all_q/*.nc')
gauge_list = list()
for file in q_pathes:
    gauge_id = file.split('/')[-1][:-3]
    if gauge_id in list(gauges['gauge_id']):
        gauge_list.append(file)
        
# add train
file_rewriter(q_pathes=gauge_list,
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])

if torch.cuda.is_available():
    start_run(config_file=Path("./poor_perfom_configs/era5L_kavkaz_qmm.yml"))
gc.collect()

2023-04-10 06:04:54,170: Logging to runs_bad_regions/era5L_kavkaz_1004_060454/output.log initialized.
2023-04-10 06:04:54,171: ### Folder structure created at runs_bad_regions/era5L_kavkaz_1004_060454
2023-04-10 06:04:54,171: ### Run configurations for era5L_kavkaz
2023-04-10 06:04:54,171: experiment_name: era5L_kavkaz
2023-04-10 06:04:54,172: run_dir: runs_bad_regions/era5L_kavkaz_1004_060454
2023-04-10 06:04:54,172: train_basin_file: openf_basins.txt
2023-04-10 06:04:54,173: validation_basin_file: openf_basins.txt
2023-04-10 06:04:54,173: test_basin_file: openf_basins.txt
2023-04-10 06:04:54,173: train_start_date: 2009-01-01 00:00:00
2023-04-10 06:04:54,174: train_end_date: 2016-12-31 00:00:00
2023-04-10 06:04:54,174: validation_start_date: 2017-01-01 00:00:00
2023-04-10 06:04:54,174: validation_end_date: 2018-12-31 00:00:00
2023-04-10 06:04:54,175: test_start_date: 2019-01-01 00:00:00
2023-04-10 06:04:54,175: test_end_date: 2020-12-31 00:00:00
2023-04-10 06:04:54,175: per_basin_trai

/opt/conda/lib/python3.9/site-packages/neuralhydrology/datasetzoo/basedataset.py:460: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1666643016022/work/torch/csrc/utils/tensor_new.cpp:230.)
  per_basin_target_stds = torch.tensor([np.nanstd(obs, axis=1)], dtype=torch.float32)


# Epoch 1: 100%|██████████| 274/274 [00:09<00:00, 28.06it/s, Loss: 0.0443]
2023-04-10 06:05:08,684: Epoch 1 average loss: 0.14049608622045412
# Epoch 2: 100%|██████████| 274/274 [00:09<00:00, 29.20it/s, Loss: 0.0867]
2023-04-10 06:05:18,069: Epoch 2 average loss: 0.05810246545903004
# Epoch 3: 100%|██████████| 274/274 [00:08<00:00, 30.46it/s, Loss: 0.0509]
2023-04-10 06:05:27,068: Epoch 3 average loss: 0.054747352838842536
# Validation: 100%|██████████| 24/24 [00:02<00:00,  9.32it/s]
2023-04-10 06:05:29,662: Stored results at runs_bad_regions/era5L_kavkaz_1004_060454/validation/model_epoch003/validation_results.p
2023-04-10 06:05:29,663: Epoch 3 average validation loss: 0.21241 -- Median validation metrics: NSE: -0.20049, KGE: -0.16853
# Epoch 4: 100%|██████████| 274/274 [00:09<00:00, 30.22it/s, Loss: 0.0405]
2023-04-10 06:05:38,733: Epoch 4 average loss: 0.05252450480921208
# Epoch 5: 100%|██████████| 274/274 [00:09<00:00, 28.86it/s, Loss: 0.0899]
2023-04-10 06:05:48,230: Epoch 5 aver

28

#### Ural

In [12]:
gauges = gpd.read_file(
    '../paper_visualisation/poor_gauges/ural.gpkg')
# add test
q_pathes = glob.glob('../geo_data/great_db/nc_all_q/*.nc')
gauge_list = list()
for file in q_pathes:
    gauge_id = file.split('/')[-1][:-3]
    if gauge_id in list(gauges['gauge_id']):
        gauge_list.append(file)
        
# add train
file_rewriter(q_pathes=gauge_list,
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])

if torch.cuda.is_available():
    start_run(config_file=Path("./poor_perfom_configs/era5L_ural_qmm.yml"))
gc.collect()

2023-04-10 06:09:49,184: PROJ: internal_proj_identify: /opt/conda/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
2023-04-10 06:09:50,795: Logging to runs_bad_regions/era5L_ural_1004_060950/output.log initialized.
2023-04-10 06:09:50,796: ### Folder structure created at runs_bad_regions/era5L_ural_1004_060950
2023-04-10 06:09:50,796: ### Run configurations for era5L_ural
2023-04-10 06:09:50,796: experiment_name: era5L_ural
2023-04-10 06:09:50,796: run_dir: runs_bad_regions/era5L_ural_1004_060950
2023-04-10 06:09:50,797: train_basin_file: openf_basins.txt
2023-04-10 06:09:50,797: validation_basin_file: openf_basins.txt
2023-04-10 06:09:50,798: test_basin_file: openf_basins.txt
2023-04-10 06:09:50,798: train_start_date: 2009-01-01 00:00:00
2023-04-10 06:09:50,799: train_end_date: 2016-12-31 00:00:00
2023-04-10 06:09:50,799: validation_start_date: 2017-01-01 00:00:00
2023-04-10 06:09:50,800: validati

28

#### VolgoDon

In [13]:
gauges = gpd.read_file(
    '../paper_visualisation/poor_gauges/volgo_don.gpkg')
# add test
q_pathes = glob.glob('../geo_data/great_db/nc_all_q/*.nc')
gauge_list = list()
for file in q_pathes:
    gauge_id = file.split('/')[-1][:-3]
    if gauge_id in list(gauges['gauge_id']):
        gauge_list.append(file)
        
# add train
file_rewriter(q_pathes=gauge_list,
              ts_dir=ts_dir,
              hydro_target='q_mm_day',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])

if torch.cuda.is_available():
    start_run(config_file=Path("./poor_perfom_configs/era5L_volgoDon_qmm.yml"))
gc.collect()

2023-04-10 06:18:15,646: PROJ: internal_proj_identify: /opt/conda/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
2023-04-10 06:18:19,207: Logging to runs_bad_regions/era5L_volgoDon_1004_061819/output.log initialized.
2023-04-10 06:18:19,208: ### Folder structure created at runs_bad_regions/era5L_volgoDon_1004_061819
2023-04-10 06:18:19,208: ### Run configurations for era5L_volgoDon
2023-04-10 06:18:19,208: experiment_name: era5L_volgoDon
2023-04-10 06:18:19,209: run_dir: runs_bad_regions/era5L_volgoDon_1004_061819
2023-04-10 06:18:19,209: train_basin_file: openf_basins.txt
2023-04-10 06:18:19,209: validation_basin_file: openf_basins.txt
2023-04-10 06:18:19,210: test_basin_file: openf_basins.txt
2023-04-10 06:18:19,210: train_start_date: 2009-01-01 00:00:00
2023-04-10 06:18:19,210: train_end_date: 2016-12-31 00:00:00
2023-04-10 06:18:19,211: validation_start_date: 2017-01-01 00:00:00
2023-04-10 06

0